<a href="https://colab.research.google.com/github/wandiab/PINNs_BuckleyLeverett/blob/main/BuckleyLeverett_DataFree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install pyDOE

In [2]:
import sys
import json
import os
import tensorflow as tf
import numpy as np
import tensorflow.experimental.numpy as tnp
import tensorflow_probability as tfp
import scipy
from scipy import io
import random
import scipy.io
import time
from datetime import datetime
from pyDOE import lhs

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy.interpolate import griddata
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import matplotlib.pyplot as plt

random.seed(1234)
# Manually making sure the numpy random seeds are "the same" on all devices
np.random.seed(1234)
tf.random.set_seed(1234)

Plotting & Logger

In [3]:

mpl.use('pgf')

def figsize(scale, nplots = 1):
    fig_width_pt = 690.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = nplots*fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# I make my own newfig and savefig functions
def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots), dpi=200)
    ax = fig.add_subplot(111)
    return fig, ax

def plot_inf_cont_results(X_star, u_pred, X_u_train, u_train, Exact_u, X, T, x, t):

  # Interpolating the results on the whole (x,t) domain.
  # griddata(points, values, points at which to interpolate, method)
  U_pred = griddata(X_star, u_pred, (X, T), method='cubic')

  # Creating the figures
  fig, ax = newfig(1.0, 1.1)
  ax.axis('off')

  ####### Row 0: u(t,x) ##################    
  gs0 = gridspec.GridSpec(1, 1)
  gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
  ax = plt.subplot(gs0[:, :])

  h = ax.imshow(U_pred.T, interpolation='nearest', cmap='viridis', 
                extent=[t.min(), t.max(), x.min(), x.max()], 
                origin='lower', aspect='auto')
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.05)
  fig.colorbar(h, cax=cax)

  ax.plot(X_u_train[:,1], X_u_train[:,0], 'kx', label = 'Data (%d points)' % (u_train.shape[0]), markersize = 4, clip_on = False)

  line = np.linspace(x.min(), x.max(), 2)[:,None]
  ax.plot(t[10]*np.ones((2,1)), line, 'r--', linewidth = 1)
  ax.plot(t[25]*np.ones((2,1)), line, 'r--', linewidth = 1)
  ax.plot(t[40]*np.ones((2,1)), line, 'r--', linewidth = 1)
  ax.plot(t[55]*np.ones((2,1)), line, 'r--', linewidth = 1)
  ax.plot(t[70]*np.ones((2,1)), line, 'r--', linewidth = 1)
  ax.plot(t[90]*np.ones((2,1)), line, 'r--', linewidth = 1)    

  ax.set_xlabel('$t$')
  ax.set_ylabel('$x$')
  ax.legend(frameon=False, loc = 'best')
  ax.set_title('$u(t,x)$', fontsize = 5)

  ####### Row 1: u(t,x) slices ##################    
  gs1 = gridspec.GridSpec(2, 5)
  # Change top=1-1/4 to top=1-1/3 to get both plots at the same time 
  gs1.update(top=1-1/4, bottom=0.2, left=0.1, right=0.9, wspace=0.5)

  ax = plt.subplot(gs1[0, 0])
  ax.plot(x,Exact_u[10,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[10,:], 'r--', linewidth = 1.5, label = 'Prediction')
  ax.set_xlabel('$x$')
  ax.set_ylabel('$u(t,x)$')    
  ax.set_title('$t = 0.10$', fontsize = 7)
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])

  ax = plt.subplot(gs1[0, 1])
  ax.plot(x,Exact_u[25,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[25,:], 'r--', linewidth = 1.5, label = 'Prediction')
  ax.set_title('$t = 0.25$', fontsize = 7)
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])

  ax = plt.subplot(gs1[0, 2])
  ax.plot(x,Exact_u[40,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[40,:], 'r--', linewidth = 1.5, label = 'Prediction')
  ax.set_title('$t = 0.40$', fontsize = 7)
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])

  ax = plt.subplot(gs1[1, 0])
  ax.plot(x,Exact_u[55,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[55,:], 'r--', linewidth = 1.5, label = 'Prediction')
  ax.set_xlabel('$x$')
  ax.set_ylabel('$u(t,x)$')
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])
  ax.set_title('$t = 0.55$', fontsize = 7)
  ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.35), ncol=5, frameon=False)

  ax = plt.subplot(gs1[1, 1])
  ax.plot(x,Exact_u[70,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[70,:], 'r--', linewidth = 1.5, label = 'Prediction')  
  ax.set_title('$t = 0.70$', fontsize = 7)
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])

  ax = plt.subplot(gs1[1, 2])
  ax.plot(x,Exact_u[90,:], color='blue', linewidth = 1.5, label = 'Exact')       
  ax.plot(x,U_pred[90,:], 'r--', linewidth = 1.5, label = 'Prediction')
  ax.axis('square')
  ax.set_xlim([-0.1,1.1])
  ax.set_ylim([-0.1,1.1])    
  ax.set_title('$t = 0.90$', fontsize = 7)

  plt.show();

class Logger(object):
  def __init__(self, frequency=20):
    print("TensorFlow version: {}".format(tf.__version__))
    print("Eager execution: {}".format(tf.executing_eagerly()))
    print("GPU-accerelated: {}".format(tf.test.is_gpu_available()))

    self.start_time = time.time()
    self.frequency = frequency

  def __get_elapsed(self):
    return datetime.fromtimestamp(time.time() - self.start_time).strftime("%M:%S")

  def __get_error_u(self):
    return self.error_fn()

  def set_error_fn(self, error_fn):
    self.error_fn = error_fn
  
  def log_train_start(self, model):
    print("\nTraining started")
    print("================")
    #self.model = model
    #print(self.model.summary())

  def log_train_epoch(self, epoch, loss, custom="", is_iter=False):
    if epoch % self.frequency == 0:
      print(f"{'nt_epoch' if is_iter else 'tf_epoch'} = {epoch:6d}  elapsed = {self.__get_elapsed()}  loss = {loss:.4e} " + custom)

  def log_train_opt(self, name):
    # print(f"tf_epoch =      0  elapsed = 00:00  loss = 2.7391e-01  error = 9.0843e-01")
    print(f"—— Starting {name} optimization ——")

  def log_train_end(self, epoch, custom=""):
    print("==================")
    print(f"Training finished (epoch {epoch}): duration = {self.__get_elapsed()}" + custom)

custom_lbfgs.py

In [4]:
# Adapted from https://github.com/yaroslavvb/stuff/blob/master/eager_lbfgs/eager_lbfgs.py

import tensorflow as tf
import numpy as np
import time

# Time tracking functions
global_time_list = []
global_last_time = 0
def reset_time():
  global global_time_list, global_last_time
  global_time_list = []
  global_last_time = time.perf_counter()
  
def record_time():
  global global_last_time, global_time_list
  new_time = time.perf_counter()
  global_time_list.append(new_time - global_last_time)
  global_last_time = time.perf_counter()
  #print("step: %.2f"%(global_time_list[-1]*1000))

def last_time():
  """Returns last interval records in millis."""
  global global_last_time, global_time_list
  if global_time_list:
    return 1000 * global_time_list[-1]
  else:
    return 0

def dot(a, b):
  """Dot product function since TensorFlow doesn't have one."""
  return tf.reduce_sum(a*b)

def verbose_func(s):
  print(s)

final_loss = None
times = []
def lbfgs(opfunc, x, config, state, do_verbose, log_fn):
  """port of lbfgs.lua, using TensorFlow eager mode.
  """

  if config.maxIter == 0:
    return

  global final_loss, times
  
  maxIter = config.maxIter
  maxEval = config.maxEval or maxIter*1.25
  tolFun = config.tolFun or 1e-18
  tolX = config.tolX or 1e-19
  nCorrection = config.nCorrection or 100
  lineSearch = config.lineSearch
  lineSearchOpts = config.lineSearchOptions
  learningRate = config.learningRate or 0.001
  isverbose = config.verbose or False

  # verbose function
  if isverbose:
    verbose = verbose_func
  else:
    verbose = lambda x: None

    # evaluate initial f(x) and df/dx
  f, g = opfunc(x)

  f_hist = [f]
  currentFuncEval = 1
  state.funcEval = state.funcEval + 1
  p = g.shape[0]

  # check optimality of initial point
  tmp1 = tf.abs(g)
  if tf.reduce_sum(tmp1) <= tolFun:
    verbose("optimality condition below tolFun")
    return x, f_hist

  # optimize for a max of maxIter iterations
  nIter = 0
  times = []
  while nIter < maxIter:
    start_time = time.time()
    
    # keep track of nb of iterations
    nIter = nIter + 1
    state.nIter = state.nIter + 1

    ############################################################
    ## compute gradient descent direction
    ############################################################
    if state.nIter == 1:
      d = -g
      old_dirs = []
      old_stps = []
      Hdiag = 1
    else:
      # do lbfgs update (update memory)
      y = g - g_old
      s = d*t
      ys = dot(y, s)
      
      if ys > 1e-10:
        # updating memory
        if len(old_dirs) == nCorrection:
          # shift history by one (limited-memory)
          del old_dirs[0]
          del old_stps[0]

        # store new direction/step
        old_dirs.append(s)
        old_stps.append(y)

        # update scale of initial Hessian approximation
        Hdiag = ys/dot(y, y)

      # compute the approximate (L-BFGS) inverse Hessian 
      # multiplied by the gradient
      k = len(old_dirs)

      # need to be accessed element-by-element, so don't re-type tensor:
      ro = [0]*nCorrection
      for i in range(k):
        ro[i] = 1/dot(old_stps[i], old_dirs[i])
        

      # iteration in L-BFGS loop collapsed to use just one buffer
      # need to be accessed element-by-element, so don't re-type tensor:
      al = [0]*nCorrection

      q = -g
      for i in range(k-1, -1, -1):
        al[i] = dot(old_dirs[i], q) * ro[i]
        q = q - al[i]*old_stps[i]

      # multiply by initial Hessian
      r = q*Hdiag
      for i in range(k):
        be_i = dot(old_stps[i], r) * ro[i]
        r += (al[i]-be_i)*old_dirs[i]
        
      d = r
      # final direction is in r/d (same object)

    g_old = g
    f_old = f
    
    ############################################################
    ## compute step length
    ############################################################
    # directional derivative
    gtd = dot(g, d)

    # check that progress can be made along that direction
    if gtd > -tolX:
      verbose("Can not make progress along direction.")
      break

    # reset initial guess for step size
    if state.nIter == 1:
      tmp1 = tf.abs(g)
      t = min(1, 1/tf.reduce_sum(tmp1))
    else:
      t = learningRate


    # optional line search: user function
    lsFuncEval = 0
    if lineSearch and isinstance(lineSearch) == types.FunctionType:
      # perform line search, using user function
      f,g,x,t,lsFuncEval = lineSearch(opfunc,x,t,d,f,g,gtd,lineSearchOpts)
      f_hist.append(f)
    else:
      # no line search, simply move with fixed-step
      x += t*d
      
      if nIter != maxIter:
        # re-evaluate function only if not in last iteration
        # the reason we do this: in a stochastic setting,
        # no use to re-evaluate that function here
        f, g = opfunc(x)
        lsFuncEval = 1
        f_hist.append(f)


    # update func eval
    currentFuncEval = currentFuncEval + lsFuncEval
    state.funcEval = state.funcEval + lsFuncEval

    ############################################################
    ## check conditions
    ############################################################
    if nIter == maxIter:
      break

    if currentFuncEval >= maxEval:
      # max nb of function evals
      verbose('max nb of function evals')
      break

    tmp1 = tf.abs(g)
    if tf.reduce_sum(tmp1) <=tolFun:
      # check optimality
      verbose('optimality condition below tolFun')
      break
    
    tmp1 = tf.abs(d*t)
    if tf.reduce_sum(tmp1) <= tolX:
      # step size below tolX
      verbose('step size below tolX')
      break

    if tf.abs(f-f_old) < tolX:
      # function value changing less than tolX
      verbose('function value changing less than tolX'+str(tf.abs(f-f_old)))
      break

    if do_verbose:
      log_fn(nIter, f.numpy(), True)
      #print("Step %3d loss %6.5f msec %6.3f"%(nIter, f.numpy(), last_time()))
      record_time()
      times.append(last_time())

    if nIter == maxIter - 1:
      final_loss = f.numpy()


  # save state
  state.old_dirs = old_dirs
  state.old_stps = old_stps
  state.Hdiag = Hdiag
  state.g_old = g_old
  state.f_old = f_old
  state.t = t
  state.d = d

  return x, f_hist, currentFuncEval

# dummy/Struct gives Lua-like struct object with 0 defaults
class dummy(object):
  pass

class Struct(dummy):
  def __getattribute__(self, key):
    if key == '__dict__':
      return super(dummy, self).__getattribute__('__dict__')
    return self.__dict__.get(key, 0)

# Prep Data

In [5]:
def prep_data(path, N_u=None, N_f=None):
    # Reading external data [t is 100x1, usol is 256x100 (solution), x is 256x1]
    #data = scipy.io.loadmat(path)
    data = path
    # Flatten makes [[]] into [], [:,None] makes it a column vector
    t = data['t'].flatten()[:,None] # T x 1
    x = data['x'].flatten()[:,None] # N x 1

    # Keeping the 2D data for the solution data (real() is maybe to make it float by default, in case of zeroes)
    Exact_u = np.real(data['usol']).T # T x N

    # Meshing x and t in 2D (256,100)
    X, T = np.meshgrid(x,t)

    # Preparing the inputs x and t (meshed as X, T) for predictions in one single array, as X_star
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))

    # Preparing the testing u_star
    u_star = Exact_u.flatten()[:,None]
                
    # Noiseless data TODO: add support for noisy data    
    idx = np.random.choice(X_star.shape[0], N_u, replace=False)
    X_u_train = X_star[idx,:]
    u_train = u_star[idx,:]

    # Domain bounds (lowerbounds upperbounds) [x, t], which are here ([-1.0, 0.0] and [1.0, 1.0])
    lb = X_star.min(axis=0)
    ub = X_star.max(axis=0) 
    # Getting the initial conditions (t=0)
    xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
    uu1 = Exact_u[0:1,:].T
    # Getting the lowest boundary conditions (x=0) 
    xx2 = np.hstack((X[:,0:1], T[:,0:1]))
    uu2 = Exact_u[:,0:1]

    # Stacking them in multidimensional tensors for training (X_u_train is for now the continuous boundaries)
    X_u_train = np.vstack([xx1, xx2])
    u_train = np.vstack([uu1, uu2])

    # Generating the x and t collocation points for f, with each having a N_f size
    # We pointwise add and multiply to spread the LHS over the 2D domain
    X_f_train = lb + (ub-lb)*lhs(2, N_f)

    # Generating a uniform random sample from ints between 0, and the size of x_u_train, of size N_u (initial data size) and without replacement (unique)
    idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
    # Getting the corresponding X_u_train (which is now scarce boundary/initial coordinates)
    X_u_train = X_u_train[idx,:]
    # Getting the corresponding u_train
    u_train = u_train [idx,:]

    return x, t, X, T, Exact_u, X_star, u_star, X_u_train, u_train, X_f_train, ub, lb

# HYPER PARAMETERS

In [7]:
# Data size on the solution u
N_u = 300
N_L = 5000
N_i = 5000
# Collocation points size, where we’ll check for f = 0
N_f = 90000

mode = 'M1' #Use M1 MLP, M2 for attention

# Getting the data
path = scipy.io.loadmat('/content/Buckley_Swc_0_Sor_0_M_2.mat');
x, t, X, T, Exact_u, X_star, u_star, X_u_train, u_train, X_f, ub, lb = prep_data(path, N_u, N_f)

IC = lb + np.array([1.0, 0.0]) * lhs(2, N_i)
Lb = lb + np.array([0.0, 1.0]) * lhs(2, N_L)
S_bc_v = np.zeros((N_L, 1)) + 1
S_ic_v = np.zeros((N_L, 1))

# DeepNN topology (2-sized input [x t], 8 hidden layer of 20-width, 1-sized output [u]
layers_part = [2, 20, 20, 20, 20, 1]
layers_Dist = [2, 20, 20, 20, 20, 20, 1]
layers = [2, 20, 20, 20, 20, 1]

# Exponential Decaying learing rate
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=False)

# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
tf_epochs = 150000 
tf_optimizer = tf.keras.optimizers.Adam(
  learning_rate= lr_schedule,
  beta_1=0.90,
  epsilon=1e-7)

# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
nt_epochs = 0
nt_config = Struct()
nt_config.learningRate = 1e-2
nt_config.maxIter = nt_epochs
nt_config.nCorrection = 100
nt_config.tolFun = 1.0*np.finfo(float).eps

# Physics Informed NN

In [8]:
class PhysicsInformedNN(object):
  def __init__(self, layers_part, layers_Dist, layers, optimizer, logger, X_f, IC, Lb, S_bc_v, S_ic_v, DIST, ub, lb, mode):
    self.mode = mode
    self.layers = layers

    self.weights_S, self.biases_S = self.initialize_NN_S(layers)
    
    if mode in ['M2']:
      # Initialize encoder weights and biases
      self.encoder_weights_1_S = self.xavier_init([2, layers[1]])  
      self.encoder_biases_1_S = self.xavier_init([1, layers[1]])

      self.encoder_weights_2_S = self.xavier_init([2, layers[1]])
      self.encoder_biases_2_S = self.xavier_init([1, layers[1]])

    # Descriptive Keras model [2, 20, …, 20, 2] 
    self.Dist_model = tf.keras.Sequential()
    self.Dist_model.add(tf.keras.layers.InputLayer(input_shape=(layers_Dist[0],)))
    self.Dist_model.add(tf.keras.layers.Lambda(lambda X: 1.0*(X - lb)/(ub - lb) - 0.0))
    for width in layers_Dist[1:-1]:
        self.Dist_model.add(tf.keras.layers.Dense(width, activation=tf.nn.relu, kernel_initializer='glorot_normal'))
    self.Dist_model.add(tf.keras.layers.Dense(layers_Dist[-1], activation=tf.nn.relu, kernel_initializer='glorot_normal')) 

    # Descriptive Keras model [2, 20, …, 20, 2]
    self.part_model = tf.keras.Sequential()
    self.part_model.add(tf.keras.layers.InputLayer(input_shape=(layers_part[0],)))
    self.part_model.add(tf.keras.layers.Lambda(lambda X: X))
    for width in layers_part[1:-1]:
        self.part_model.add(tf.keras.layers.Dense(width, activation=tf.nn.tanh, kernel_initializer='glorot_normal'))
    self.part_model.add(tf.keras.layers.Dense(layers_part[-1], activation=tf.nn.sigmoid, kernel_initializer='glorot_normal'))

    # Computing the sizes of weights/biases for future decomposition
    self.sizes_w = []
    self.sizes_b = []
    for i, width in enumerate(layers):
      if i != 1:
        self.sizes_w.append(int(width * layers[1]))
        self.sizes_b.append(int(width if i != 0 else layers[1]))

    # Computing the sizes of weights/biases for future decomposition - particular solution
    self.sizes_w_part = []
    self.sizes_b_part = []
    for i, width in enumerate(layers_part):
      if i != 1:
        self.sizes_w_part.append(int(width * layers_part[1]))
        self.sizes_b_part.append(int(width if i != 0 else layers_part[1]))

    # Computing the sizes of weights/biases for future decomposition - particular solution
    self.sizes_w_Dist = []
    self.sizes_b_Dist = []
    for i, width in enumerate(layers_Dist):
      if i != 1:
        self.sizes_w_Dist.append(int(width * layers_Dist[1]))
        self.sizes_b_Dist.append(int(width if i != 0 else layers_Dist[1]))

    self.optimizer = optimizer
    self.logger = logger
    self.dtype = tf.float32

    # Separating the collocation coordinates
    self.x_f = tf.convert_to_tensor(X_f[:, 0:1], dtype=self.dtype)
    self.t_f = tf.convert_to_tensor(X_f[:, 1:2], dtype=self.dtype)

  # Defining custom loss for particular solution network
  @tf.function
  def __loss_part(self, x_IC, t_IC, x_Lb, t_Lb, S_ic_v, S_bc_v):
    S_i = self.part_model(tf.stack([x_IC, t_IC], axis=1))
    S_bc = self.part_model(tf.stack([x_Lb, t_Lb], axis=1))    
    return tf.reduce_mean(tf.square(S_i)) + tf.reduce_mean(tf.square(S_bc_v - S_bc))

  # Defining custom loss for distance function network
  @tf.function
  def __loss_Dist(self, x_dist, t_dist, S_dist):
    D_S = self.Dist_model(tf.stack([x_dist, t_dist], axis=1))
    return tf.reduce_mean(tf.square(D_S - S_dist))
    
  # Defining custom loss
  @tf.function
  def __loss(self):
    f_pred = self.f_model()
    return tf.reduce_mean(tf.square(f_pred))

  # Xavier initialization
  def xavier_init(self, size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = 1. / np.sqrt((in_dim + out_dim) / 2.)
    return tf.Variable(tf.random.normal([in_dim, out_dim], dtype=tf.float32) * xavier_stddev,
                       dtype=tf.float32)
    
  # Initialize S network weights and biases using Xavier initialization
  def initialize_NN_S(self, layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
      W = self.xavier_init(size=[layers[l], layers[l + 1]])
      b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32), dtype=tf.float32)
      weights.append(W)
      biases.append(b)
    return weights, biases

  # Evaluates the forward pass.
  @tf.function
  def forward_pass_S(self, H):
    H = 1.0*(H - lb)/(ub - lb) - 0.0
    if self.mode in ['M1']:
      num_layers = len(self.layers)
      for l in range(0, num_layers - 2):
        W = self.weights_S[l]
        b = self.biases_S[l]
        H = tf.tanh(tf.add(tf.matmul(H, W), b))
      W = self.weights_S[-1]
      b = self.biases_S[-1]
      H = tf.add(tf.matmul(H, W), b)
      return H
    if self.mode in ['M2']:
      num_layers = len(self.layers)
      encoder_1 = tf.tanh(tf.add(tf.matmul(H, self.encoder_weights_1_S), self.encoder_biases_1_S))
      encoder_2 = tf.tanh(tf.add(tf.matmul(H, self.encoder_weights_2_S), self.encoder_biases_2_S))
      for l in range(0, num_layers - 2):
        W = self.weights_S[l]
        b = self.biases_S[l]
        H = tf.math.multiply(tf.tanh(tf.add(tf.matmul(H, W), b)), encoder_1) + \
        tf.math.multiply(tf.tanh(tf.add(1 - tf.matmul(H, W), b)), encoder_2)
      W = self.weights_S[-1]
      b = self.biases_S[-1]
      H = tf.add(tf.matmul(H, W), b)
      return H

  @tf.function
  def net_S(self, x, t):
    u = self.forward_pass_S(tf.stack([x[:,0], t[:,0]], axis=1))
    part = self.part_model(tf.stack([x[:,0], t[:,0]], axis=1))
    dist = self.Dist_model(tf.stack([x[:,0], t[:,0]], axis=1))
    S = part + (dist * u)
    return S

  # The actual PINN
  @tf.function
  def f_model(self):    
    with tf.GradientTape(persistent=True) as tape:
      tape.watch(self.x_f)
      tape.watch(self.t_f)
      S = self.net_S(self.x_f, self.t_f)

      Swc = 0.0
      M = 2
      Sor = 0.0
      #Non-Convex Flux Function
      frac_org = tf.divide(tf.square(S-Swc), tf.square(S-Swc) + tf.divide(tf.square(1 - S - Sor), M))
      Sf = tf.sqrt(tf.divide(1/M, (1/M)+1))
      frac_Sf = tf.divide(tf.square(Sf-Swc), tf.square(Sf-Swc) + tf.divide(tf.square(1 - Sf - Sor), M))
      frac = tf.divide(frac_Sf, Sf)*S - tf.multiply(tf.divide(frac_Sf, Sf)*S, tnp.heaviside(S-Sf, 1)) + tf.multiply(frac_org, tnp.heaviside(S-Sf, 1)) 
    S_x = tape.gradient(S, self.x_f)
    S_t = tape.gradient(S, self.t_f)
    frac_S = tape.gradient(frac, S)
    del tape 
    return S_t + tf.multiply(frac_S, S_x)

  def get_weights_model(self, model):
    w = []
    for layer in model.layers[1:]:
      weights_biases = layer.get_weights()
      weights = weights_biases[0].flatten()
      biases = weights_biases[1]
      w.extend(weights)
      w.extend(biases)
    return tf.convert_to_tensor(w, dtype=self.dtype)

  def get_weights(self):
    w_S = []
    for i in range(len(self.weights_S)):
      W_S = self.weights_S[i]
      b_S = self.biases_S[i]
      w_S.append(W_S)
      w_S.append(b_S)
    return w_S

  def get_weights_lbfgs(self):
    self.w_S = []
    for i in range(len(self.weights_S)):
      W_S = np.array(self.weights_S[i]).flatten()
      b_S = np.array(self.biases_S[i]).flatten()
      self.w_S.extend(W_S)
      self.w_S.extend(b_S)
    w = self.w_S
    return tf.convert_to_tensor(w, dtype=tf.float32)

  def set_weights(self, w):
    w_S = w[0:len(self.w_S)]
    weights1_S = []
    biases1_S = []
    for i, q in enumerate(self.layers[1:]):
      start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
      end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
      weights_S = w_S[start_weights:end_weights]
      w_div = int(self.sizes_w[i] / self.sizes_b[i])
      weights_S = tf.reshape(weights_S, [w_div, self.sizes_b[i]])
      biases_S = w_S[end_weights:end_weights + self.sizes_b[i]]
      biases_S = tf.reshape(biases_S, [1, self.sizes_b[i]])  
      weights1_S.append(tf.Variable(weights_S))
      biases1_S.append(tf.Variable(biases_S))
      if np.array(weights1_S).shape == ((len(layers)-1),):
        self.weights_S = weights1_S
        self.biases_S = biases1_S

  def set_weights_part(self, w):
    for i, layer in enumerate(self.part_model.layers[1:]):
      start_weights = sum(self.sizes_w_part[:i]) + sum(self.sizes_b_part[:i])
      end_weights = sum(self.sizes_w_part[:i+1]) + sum(self.sizes_b_part[:i])
      weights = w[start_weights:end_weights]
      w_div = int(self.sizes_w_part[i] / self.sizes_b_part[i])
      weights = tf.reshape(weights, [w_div, self.sizes_b_part[i]])
      biases = w[end_weights:end_weights + self.sizes_b_part[i]]
      weights_biases = [weights, biases]
      layer.set_weights(weights_biases)

  def set_weights_Dist(self, w):
    for i, layer in enumerate(self.Dist_model.layers[1:]):
      start_weights = sum(self.sizes_w_Dist[:i]) + sum(self.sizes_b_Dist[:i])
      end_weights = sum(self.sizes_w_Dist[:i+1]) + sum(self.sizes_b_Dist[:i])
      weights = w[start_weights:end_weights]
      w_div = int(self.sizes_w_Dist[i] / self.sizes_b_Dist[i])
      weights = tf.reshape(weights, [w_div, self.sizes_b_Dist[i]])
      biases = w[end_weights:end_weights + self.sizes_b_Dist[i]]
      weights_biases = [weights, biases]
      layer.set_weights(weights_biases)

  # The training function
  def fit_part(self, tf_epochs, nt_config):
    self.logger.log_train_start(self)

    #Left Boundary 
    self.x_Lb = tf.convert_to_tensor(Lb[:, 0:1], dtype=self.dtype)
    self.t_Lb = tf.convert_to_tensor(Lb[:, 1:2], dtype=self.dtype)
    self.S_bc_v = tf.convert_to_tensor(S_bc_v, dtype=self.dtype)

    # Initial condition point, t=0
    self.x_IC = tf.convert_to_tensor(IC[:, 0:1], dtype=self.dtype)
    self.t_IC = tf.convert_to_tensor(IC[:, 1:2], dtype=self.dtype)
    self.S_ic_v = tf.convert_to_tensor(S_ic_v, dtype=self.dtype)

    self.logger.log_train_opt("Adam")
    for epoch in range(tf_epochs):
      # Optimization step
      with tf.GradientTape() as tape:
        loss_part = self.__loss_part(self.x_IC, self.t_IC, self.x_Lb, self.t_Lb, self.S_ic_v, self.S_bc_v)
      grad = tape.gradient(loss_part, self.part_model.trainable_variables)
      self.optimizer.apply_gradients(zip(grad, self.part_model.trainable_variables))
      self.logger.log_train_epoch(epoch, loss_part)

    self.logger.log_train_opt("LBFGS")
    def loss_and_flat_grad(w):
      with tf.GradientTape() as tape:
        self.set_weights_part(w)
        loss_part = self.__loss_part(self.x_IC, self.t_IC, self.x_Lb, self.t_Lb, self.S_ic_v, self.S_bc_v)
      grad = tape.gradient(loss_part, self.part_model.trainable_variables)
      
      grad_flat = []
      for g in grad:
        grad_flat.append(tf.reshape(g, [-1]))
      grad_flat =  tf.concat(grad_flat, 0)
      return loss_part, grad_flat

    lbfgs(loss_and_flat_grad,
      self.get_weights_model(self.part_model),
      nt_config, Struct(), True,
      lambda epoch, loss, is_iter:
        self.logger.log_train_epoch(epoch, loss, "", is_iter))

  # The training function
  def fit_Dist(self, tf_epochs, nt_config):
    self.logger.log_train_start(self)

    # Distance function
    self.x_dist = tf.convert_to_tensor(DIST[:, 0:1], dtype=self.dtype)
    self.t_dist = tf.convert_to_tensor(DIST[:, 1:2], dtype=self.dtype)
    self.S_dist = tf.convert_to_tensor(DIST[:, 2:3], dtype=self.dtype)

    self.logger.log_train_opt("Adam")
    for epoch in range(tf_epochs):
      # Optimization step
      with tf.GradientTape() as tape:
        loss_Dist = self.__loss_Dist(self.x_dist, self.t_dist, self.S_dist)
      grad = tape.gradient(loss_Dist, self.Dist_model.trainable_variables)
      self.optimizer.apply_gradients(zip(grad, self.Dist_model.trainable_variables))
      self.logger.log_train_epoch(epoch, loss_Dist)

    self.logger.log_train_opt("LBFGS")
    def loss_and_flat_grad(w):
      with tf.GradientTape() as tape:
        self.set_weights_Dist(w)
        loss_Dist = self.__loss_Dist(self.x_dist, self.t_dist, self.S_dist)
      grad = tape.gradient(loss_Dist, self.Dist_model.trainable_variables)
      
      grad_flat = []
      for g in grad:
        grad_flat.append(tf.reshape(g, [-1]))
      grad_flat =  tf.concat(grad_flat, 0)
      return loss_Dist, grad_flat

    lbfgs(loss_and_flat_grad,
      self.get_weights_model(self.Dist_model),
      nt_config, Struct(), True,
      lambda epoch, loss, is_iter:
        self.logger.log_train_epoch(epoch, loss, "", is_iter))

  # The training function
  def fit(self, tf_epochs, nt_config=Struct()):
    self.logger.log_train_start(self)

    self.logger.log_train_opt("Adam")
    for epoch in range(tf_epochs):
      # Optimization step
      with tf.GradientTape() as tape:
        loss_value = self.__loss()
      grad = tape.gradient(loss_value, self.get_weights())
      self.optimizer.apply_gradients(zip(grad, self.get_weights()))
      self.logger.log_train_epoch(epoch, loss_value)

    self.logger.log_train_opt("LBFGS")
    def loss_and_flat_grad(w):
      with tf.GradientTape() as tape:
        self.set_weights(w)
        loss_value = self.__loss()
      grad = tape.gradient(loss_value, self.get_weights())
      grad_flat = []
      for g in grad:
        grad_flat.append(tf.reshape(g, [-1]))
      grad_flat =  tf.concat(grad_flat, 0)
      return loss_value, grad_flat

    lbfgs(loss_and_flat_grad,
      self.get_weights_lbfgs(),
      nt_config, Struct(), True,
      lambda epoch, loss, is_iter:
        self.logger.log_train_epoch(epoch, loss, "", is_iter))
    
    self.logger.log_train_end(tf_epochs + nt_config.maxIter)

  def predict(self, X_star):
    x = tf.convert_to_tensor(X_star[:, 0:1], dtype=self.dtype)
    t = tf.convert_to_tensor(X_star[:, 1:2], dtype=self.dtype)
    S = self.net_S(x, t)
    return S

  def predict_part(self, X_star):
    x_p = tf.convert_to_tensor(X_star[:, 0:1], dtype=self.dtype)
    t_p = tf.convert_to_tensor(X_star[:, 1:2], dtype=self.dtype)
    Part_S = self.part_model(tf.stack([x_p, t_p], axis=1))
    return Part_S

  def predict_Dist(self, X_star):
    x_D = tf.convert_to_tensor(X_star[:, 0:1], dtype=self.dtype)
    t_D = tf.convert_to_tensor(X_star[:, 1:2], dtype=self.dtype)
    D_S = self.Dist_model(tf.stack([x_D, t_D], axis=1))
    return D_S

def GenDistPt(xmin, xmax, tmin, tmax, num):
    # num: number per edge
    x = np.linspace(xmin, xmax, num=num)
    t = np.linspace(tmin, tmax, num=num)
    
    xxx, ttt = np.meshgrid(x, t)
    xxx = xxx.flatten()[:, None]
    ttt = ttt.flatten()[:, None]
    return xxx, ttt

def GenDist(XYT_dist):
    dist_S = np.zeros_like(XYT_dist[:, 0:1])
    for i in range(len(XYT_dist)):
        dist_S[i, 0] = min(XYT_dist[i][0], XYT_dist[i][1])  # Initial Condition & Left Boundary 
    DIST = np.concatenate((XYT_dist, dist_S), 1)
    idx = np.random.choice(DIST.shape[0], DIST.shape[0], replace=False)
    DIST = DIST[idx,:]
    return DIST

In [9]:
# Generate distance function for spatio-temporal space
x_dist, t_dist = GenDistPt(xmin=0, xmax=1.0, tmin=0, tmax=1.0, num=50)
XYT_dist = np.concatenate((x_dist, t_dist), 1)
DIST = GenDist(XYT_dist)

# Training - Preprocessing - Distance


In [ ]:
# Training the Distance function NN
logger = Logger(frequency=100)
pinn = PhysicsInformedNN(layers_part, layers_Dist, layers, tf_optimizer, logger, X_f, IC, Lb, S_bc_v, S_ic_v, DIST, ub, lb, mode)
pinn.fit_Dist(tf_epochs, nt_config)

In [ ]:
# Saving Distance Model Parameters
Dist_Model = pinn.Dist_model
Dist_Model.save_weights('Distance_model_weights_BL.h5')

# Training - Preprocessing - *Particular*

In [ ]:
# Training the Particular Solution NN
logger = Logger(frequency=100)
pinn = PhysicsInformedNN(layers_part, layers_Dist, layers, tf_optimizer, logger, X_f, IC, Lb, S_bc_v, S_ic_v, DIST, ub, lb, mode)
pinn.fit_part(tf_epochs, nt_config)

In [ ]:
# Saving Particular Model Parameters
Part_model = pinn.part_model
Part_model.save_weights('Particular_model_weights_BL.h5')

# Training

In [ ]:
# Getting the data
path = scipy.io.loadmat('/content/Buckley_Swc_0_Sor_0_M_2.mat');
x, t, X, T, Exact_u, X_star, u_star, X_u_train, u_train, X_f, ub, lb = prep_data(path, N_u, N_f)

# Creating the model and training
logger = Logger(frequency=200)
pinn = PhysicsInformedNN(layers_part, layers_Dist, layers, tf_optimizer, logger, X_f, IC, Lb, S_bc_v, S_ic_v, DIST, ub, lb, mode)

# Loading Particular Model Parameters
Part_model = pinn.part_model
Part_model.load_weights('Particular_model_weights_BL.h5')

# Loading Distance Model Parameters
Dist_Model = pinn.Dist_model
pinn.Dist_model.load_weights('Distance_model_weights_BL.h5')

pinn.fit(tf_epochs, nt_config)

In [ ]:
# L2 Error
u_pred = pinn.predict(X_star)
print('L2 Error: %.3e' %(np.linalg.norm(u_star - u_pred) / np.linalg.norm(u_star)))

# Plotting
plot_inf_cont_results(X_star, u_pred.numpy().flatten(), X_u_train, u_train, Exact_u, X, T, x, t)